## 973. K Closest Points to Origin
- Description:
  <blockquote>
    Given an array of `points` where `points[i] = [xi, yi]` represents a point on the **X-Y** plane and an integer `k`, return the `k` closest points to the origin `(0, 0)`.
   
  The distance between two points on the **X-Y** plane is the Euclidean distance (i.e., `√(x1- x2)2+ (y1- y2)2`).
   
  You may return the answer in **any order**. The answer is **guaranteed** to be **unique** (except for the order that it is in).
   
  **Example 1:**
  ![Image](https://assets.leetcode.com/uploads/2021/03/03/closestplane1.jpg)
   
  **Input:** points = `[1, 3],[-2,2]`, k = 1
  **Output:** `[-2, 2]`
  **Explanation:**
  The distance between (1, 3) and the origin is sqrt(10).
  The distance between (-2, 2) and the origin is sqrt(8).
  Since sqrt(8) < sqrt(10), (-2, 2) is closer to the origin.
  We only want the closest k = 1 points from the origin, so the answer is just `[-2, 2]`.
   
  **Example 2:**
  **Input:** points = `[3, 3],[5,-1],[-2,4]`, k = 2
  **Output:** `[3, 3],[-2,4]`
  **Explanation:** The answer `[-2, 4],[3,3]` would also be accepted.
   
  **Constraints:**
   
  - `1 <= k <= points.length <= 104`
  - `-104<= xi, yi<= 104`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/k-closest-points-to-origin/description/)

- Topics: Max Heap

- Difficulty: Medium / Easy

- Resources: example_resource_URL

### Solution 1, Max Heap
Here N refers to the length of the given array points.

- Time complexity: O(N⋅logk)

Adding to/removing from the heap (or priority queue) only takes O(logk) time when the size of the heap is capped at k elements.

- Space complexity: O(k)

The heap (or priority queue) will contain at most k elements.


In [ ]:
class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        heap = []

        for point in points:
            # You usually avoid sqrt when comparing distances because sqrt is computationally expensive
            # Ordering is preserved without it (since sqrt is monotonic)
            # ALT - math.sqrt((x1-x2)**2+(y1-y2)**2)
            # assuming origin (0,0)
            distance = point[0]**2 + point[1]**2

            if len(heap) < k:
                heapq.heappush(heap, (-distance, point))
            else:
                heapq.heappushpop(heap, (-distance, point))
        
        return [point for (_, point) in heap]



In [ ]:
# Alt

class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        # Since heap is sorted in increasing order,
        # negate the distance to simulate max heap
        # and fill the heap with the first k elements of points
        heap = [(-self.squared_distance(points[i]), i) for i in range(k)]
        heapq.heapify(heap)
        for i in range(k, len(points)):
            dist = -self.squared_distance(points[i])
            if dist > heap[0][0]:
                # If this point is closer than the kth farthest,
                # discard the farthest point and add this one
                heapq.heappushpop(heap, (dist, i))
        
        # Return all points stored in the max heap
        return [points[i] for (_, i) in heap]
    
    def squared_distance(self, point: List[int]) -> int:
        """Calculate and return the squared Euclidean distance."""
        return point[0] ** 2 + point[1] ** 2

In [ ]:
# Binary Search

class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        # Precompute the Euclidean distance for each point
        distances = [self.euclidean_distance(point) for point in points]
        # Create a reference list of point indices
        remaining = [i for i in range(len(points))]
        # Define the initial binary search range
        low, high = 0, max(distances)
        
        # Perform a binary search of the distances
        # to find the k closest points
        closest = []
        while k:
            mid = (low + high) / 2
            closer, farther = self.split_distances(remaining, distances, mid)
            if len(closer) > k:
                # If more than k points are in the closer distances
                # then discard the farther points and continue
                remaining = closer
                high = mid
            else:
                # Add the closer points to the answer array and keep
                # searching the farther distances for the remaining points
                k -= len(closer)
                closest.extend(closer)
                remaining = farther
                low = mid
                
        # Return the k closest points using the reference indices
        return [points[i] for i in closest]

    def split_distances(self, remaining: List[int], distances: List[float],
                        mid: int) -> List[List[int]]:
        """Split the distances around the midpoint
        and return them in separate lists."""
        closer, farther = [], []
        for index in remaining:
            if distances[index] <= mid:
                closer.append(index)
            else:
                farther.append(index)
        return [closer, farther]

    def euclidean_distance(self, point: List[int]) -> float:
        """Calculate and return the squared Euclidean distance."""
        return point[0] ** 2 + point[1] ** 2

In [ ]:
# QuickSelect

class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        return self.quick_select(points, k)
    
    def quick_select(self, points: List[List[int]], k: int) -> List[List[int]]:
        """Perform the QuickSelect algorithm on the list"""
        left, right = 0, len(points) - 1
        pivot_index = len(points)
        while pivot_index != k:
            # Repeatedly partition the list
            # while narrowing in on the kth element
            pivot_index = self.partition(points, left, right)
            if pivot_index < k:
                left = pivot_index
            else:
                right = pivot_index - 1
        
        # Return the first k elements of the partially sorted list
        return points[:k]
    
    def partition(self, points: List[List[int]], left: int, right: int) -> int:
        """Partition the list around the pivot value"""
        pivot = self.choose_pivot(points, left, right)
        pivot_dist = self.squared_distance(pivot)
        while left < right:
            # Iterate through the range and swap elements to make sure
            # that all points closer than the pivot are to the left
            if self.squared_distance(points[left]) >= pivot_dist:
                points[left], points[right] = points[right], points[left]
                right -= 1
            else:
                left += 1
        
        # Ensure the left pointer is just past the end of
        # the left range then return it as the new pivotIndex
        if self.squared_distance(points[left]) < pivot_dist:
            left += 1
        return left
    
    def choose_pivot(self, points: List[List[int]], left: int, right: int) -> List[int]:
        """Choose a pivot element of the list"""
        return points[left + (right - left) // 2]
    
    def squared_distance(self, point: List[int]) -> int:
        """Calculate and return the squared Euclidean distance."""
        return point[0] ** 2 + point[1] ** 2